First notebook file for the project

In [11]:
import pandas as pd
import numpy as np
data=pd.read_csv("GlobalLandTemperaturesByCountry.csv")
data_estonia=data[data["Country"]=="Estonia"]
data_estonia.to_csv("Estonia.csv")